In [ ]:
import numpy as np
import copy
import joblib
import torch
from tensorflow.keras.applications import ResNet152
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from scipy.spatial.distance import cdist


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_data = []
train_labels = []
eval_data = []
eval_labels = []

predicted_labels = {}

for i in range(10):
  t = torch.load('./dataset/part_one_dataset/train_data/' + str(i+1) + '_train_data.tar.pth')
  train_data.append(t['data'])
  if i == 0:
    train_labels.append(t['targets'])

for i in range(10):
  t = torch.load('./dataset/part_one_dataset/eval_data/' + str(i+1) + '_eval_data.tar.pth')
  eval_data.append(t['data'])
  eval_labels.append(t['targets'])

/var/folders/15/rhh2dgys0n75gk26z6nrx13c0000gn/T/ipykernel_1124/3155242173.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t = torch.load('../dataset/part_one_dataset/tr

In [5]:
train_data_extracted = []
eval_data_extracted = []
model = ResNet152(weights='imagenet', include_top=False, pooling='avg')

def process_and_extract_features(data, model):
    extracted_features = []
    for X in data:
        X_resized = np.array([image.img_to_array(image.array_to_img(img, scale=False).resize((224, 224))) for img in X])
        X_preprocessed = preprocess_input(X_resized)
        features = model.predict(X_preprocessed, batch_size=32)
        extracted_features.append(features)
    return extracted_features


train_data_extracted = process_and_extract_features(train_data, model)
eval_data_extracted = process_and_extract_features(eval_data, model)


2024-11-26 16:09:10.751699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


79/79 ━━━━━━━━━━━━━━━━━━━━ 42s 457ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 28s 355ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 28s 352ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 28s 353ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 359ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 357ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 362ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 357ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 362ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 28s 358ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 359ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 28s 354ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 358ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 356ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 361ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 362ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 364ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 358ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 28s 353ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 357ms/step


In [7]:
print(f"Train data shape: {train_data[0].shape}")
print(f"Extracted features shape: {train_data_extracted[0].shape}")

Train data shape: (2500, 32, 32, 3)
Extracted features shape: (2500, 2048)


In [10]:
# Save the arrays
np.save('./features/Part-1/train_data_extracted.npy', train_data_extracted)
np.save('./features/Part-1/eval_data_extracted.npy', eval_data_extracted)

## Run from here if features already extracted

In [30]:
train_data_extracted = np.load('./features/Part-1/train_data_extracted.npy', allow_pickle=True)
eval_data_extracted = np.load('./features/Part-1/eval_data_extracted.npy', allow_pickle=True)

print(f"Length of Datastructure: {len(train_data_extracted)}")
print(f"Extracted features shape: {train_data_extracted[0].shape}")

Length of Datastructure: 10
Extracted features shape: (2500, 2048)


In [31]:
class LwPClassifier:
    def __init__(self, metric='euclidean'):
        self.metric = metric
        self.class_prototypes = {}

    def fit(self, X, y, weights=None, alpha=0.01):
        """Compute prototypes (mean vectors) for each class, with optional weighting and regularization."""
        X = X.reshape(X.shape[0], -1)
        
        self.classes_ = np.unique(y)
        if weights is None:
            weights = np.ones_like(y, dtype=float)

        self.class_prototypes = {
            cls: np.average(X[y == cls], axis=0, weights=weights[y == cls]) - alpha * np.mean(X, axis=0) 
            for cls in self.classes_
        }

    def predict(self, X):
        """Predict class for each sample in X."""
        X = X.reshape(X.shape[0], -1)
        prototypes = np.array([self.class_prototypes[cls] for cls in self.classes_])
        distances = cdist(X, prototypes, metric=self.metric)
        closest_prototype_idx = distances.argmin(axis=1)
        return self.classes_[closest_prototype_idx]

    def retrain(self, X, y, momentum=0.9):
        """Update prototypes with new data using adaptive learning and momentum."""
        X = X.reshape(X.shape[0], -1)
        
        for cls in np.unique(y):
            if cls in self.class_prototypes:
                old_prototype = self.class_prototypes[cls]
                new_prototype = X[y == cls].mean(axis=0)
                self.class_prototypes[cls] = momentum * old_prototype + (1 - momentum) * new_prototype
            else:
                self.class_prototypes[cls] = X[y == cls].mean(axis=0)

    def score(self, X, y):
        """Calculate accuracy of the model."""
        predictions = self.predict(X)
        return np.mean(predictions == y)


In [32]:
X_train = train_data_extracted
y_train = train_labels
X_test = eval_data_extracted

In [33]:
f1 = LwPClassifier()
f1.fit(X_train[0], y_train[0])

predicted_labels['D2'] = f1.predict(X_train[1])
print(predicted_labels['D2'])

[6 7 9 ... 8 3 0]


In [34]:
f2 = copy.deepcopy(f1)
f2.retrain(X_train[1], predicted_labels['D2'])

predicted_labels['D3'] = f2.predict(X_train[2])
print(predicted_labels['D3'])

[1 6 6 ... 1 1 7]


In [35]:
f3 = copy.deepcopy(f2)
f3.retrain(X_train[2], predicted_labels['D3'])

predicted_labels['D4'] = f3.predict(X_train[3])
print(predicted_labels['D4'])

[0 6 7 ... 3 4 7]


In [36]:
f4 = copy.deepcopy(f3)
f4.retrain(X_train[3], predicted_labels['D4'])

predicted_labels['D5'] = f4.predict(X_train[4])
print(predicted_labels['D5'])

[3 5 0 ... 9 7 1]


In [37]:
f5 = copy.deepcopy(f4)
f5.retrain(X_train[4], predicted_labels['D5'])

predicted_labels['D6'] = f5.predict(X_train[5])
print(predicted_labels['D6'])

[2 9 8 ... 0 2 2]


In [38]:
f6 = copy.deepcopy(f5)
f6.retrain(X_train[5], predicted_labels['D6'])

predicted_labels['D7'] = f6.predict(X_train[6])
print(predicted_labels['D7'])

[8 6 0 ... 6 7 7]


In [39]:
f7 = copy.deepcopy(f6)
f7.retrain(X_train[6], predicted_labels['D7'])

predicted_labels['D8'] = f7.predict(X_train[7])
print(predicted_labels['D8'])

[3 6 7 ... 7 5 9]


In [40]:
f8 = copy.deepcopy(f7)
f8.retrain(X_train[7], predicted_labels['D8'])

predicted_labels['D9'] = f8.predict(X_train[8])
print(predicted_labels['D9'])

[1 8 5 ... 8 1 8]


In [41]:
f9 = copy.deepcopy(f8)
f9.retrain(X_train[8], predicted_labels['D9'])

predicted_labels['D10'] = f9.predict(X_train[9])
print(predicted_labels['D10'])

[7 1 4 ... 6 2 6]


In [42]:
f10 = copy.deepcopy(f9)
f10.retrain(X_train[9], predicted_labels['D10'])

## Evalutation

In [ ]:
models = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10]
outputs_pasrtial = [[] for _ in range(10)]


In [44]:
accuracies_partial = [[] for _ in range(10)]

for i in range(1, 10+1):
    for j in range(1, i+1):
        model_id = i-1
        data_id = j-1
        score = models[model_id].score(eval_data_extracted[data_id], eval_labels[data_id])
        accuracies_partial[model_id].append(score)

In [45]:
for i in range(10):
    print(accuracies_partial[i])

[0.8716]
[0.8712, 0.872]
[0.8688, 0.87, 0.862]
[0.8692, 0.8676, 0.8596, 0.868]
[0.8664, 0.8636, 0.8592, 0.8684, 0.8652]
[0.8648, 0.8624, 0.8592, 0.8668, 0.8648, 0.8664]
[0.8644, 0.8616, 0.8572, 0.866, 0.864, 0.864, 0.8564]
[0.864, 0.8604, 0.856, 0.8648, 0.8632, 0.8632, 0.8564, 0.8608]
[0.8628, 0.8592, 0.856, 0.8632, 0.864, 0.8612, 0.856, 0.86, 0.8488]
[0.8604, 0.8588, 0.8564, 0.8624, 0.8632, 0.8604, 0.8544, 0.86, 0.8464, 0.8568]


## Saving models

In [29]:
for i, model in enumerate(models, start=1):
    filename = f'./models/Part-1/f{i}.joblib'
    joblib.dump(model, filename)
    print(f"Model {i} saved to {filename}")

Model 1 saved to ./models/Part-1/f1.joblib
Model 2 saved to ./models/Part-1/f2.joblib
Model 3 saved to ./models/Part-1/f3.joblib
Model 4 saved to ./models/Part-1/f4.joblib
Model 5 saved to ./models/Part-1/f5.joblib
Model 6 saved to ./models/Part-1/f6.joblib
Model 7 saved to ./models/Part-1/f7.joblib
Model 8 saved to ./models/Part-1/f8.joblib
Model 9 saved to ./models/Part-1/f9.joblib
Model 10 saved to ./models/Part-1/f10.joblib
